In [1]:
!pip install tensorflow tensorflow-datasets

  Using cached tensorflow-2.20.0-cp313-cp313-win_amd64.whl.metadata (4.6 kB)
Using cached tensorflow-2.20.0-cp313-cp313-win_amd64.whl (332.0 MB)


In [5]:
!py -3.11 -m pip install --upgrade pip
!py -3.11 -m pip install tensorflow

In [6]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ImportError: Traceback (most recent call last):
  File "C:\Users\JANANI_ABOORVAVARSHA\AppData\Local\Programs\Python\Python313\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'food101',
    split=['train', 'validation'],
    as_supervised=True,
    with_info=True
)

class_names = ds_info.features['label'].names
num_classes = ds_info.features['label'].num_classes

print("Number of classes:", num_classes)

In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds = ds_train.map(preprocess).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds  = ds_test.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False

In [ ]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=5   # Increase to 10–15 if you have GPU
)

In [ ]:
test_loss, test_acc = model.evaluate(test_ds)
print("Test Accuracy:", test_acc)


In [ ]:
calorie_dict = {
    "pizza": 266,
    "hamburger": 295,
    "french_fries": 312,
    "ice_cream": 207,
    "fried_rice": 163,
    "chicken_curry": 240,
    "apple_pie": 237,
    "sushi": 200,
    "salad": 152
}


In [ ]:
def predict_food_and_calories(image):
    img = tf.image.resize(image, IMG_SIZE)
    img = tf.expand_dims(img, axis=0) / 255.0

    predictions = model.predict(img)
    predicted_class = class_names[np.argmax(predictions)]
    
    calories = calorie_dict.get(predicted_class.lower(), "Calories not available")
    
    return predicted_class, calories


In [ ]:
for image, label in ds_test.take(1):
    food, calories = predict_food_and_calories(image)
    
    plt.imshow(image)
    plt.axis('off')
    plt.title(f"Food: {food} | Calories: {calories}")
    plt.show()
